In [1]:
# Required Packages
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt


###########################################################

# Parameter Setup
# Integration parameter 
h=1/1000  # step size    need at least 1000hz (sec)
t0=0    # initial time (sec)
tf=5    # final time (sec)

time=np.linspace(t0,tf,int((tf-t0)/h)+1)

## States
#1. Hose / 2. Fitting /3. Main Valve/4. Nozzle
# Initial pressure (Pa)
p_s=11e5            #p_s pressure supply: at the inlet of the hose   
p1_t0=p_s           # inlet of the fitting
p2_t0=p_s           # inlet of the valve
p3_t0=1e5           # inlet of the nozzle:  assume initial constant air pressure

# Initial mass flow rate (kg/s)
md1_t0=0
md2_t0=0
md3_t0=0
md4_t0=0

state_t0=np.array([p1_t0,md1_t0,p2_t0,md2_t0,p3_t0,md3_t0,md4_t0])
state_t=np.zeros((len(state_t0),len(time)))


# Initial density (assuming nitrogen at T=273K)
rho_1_t0 = 1.2336 * p_s / 1e5
rho_2_t0 = 1.2336 * p1_t0 / 1e5
rho_3_t0 = 1.2336 * p2_t0 / 1e5


rho_t0=np.array([rho_1_t0,rho_2_t0,rho_3_t0])
rho_t=np.zeros((len(rho_t0),len(time)))

###########################################################

# Motion (dynamic system)
x_t0=0.0
v_t0=0.0
a_t0=0.0

x_target=5.0         # height target [m]

motion_t0=np.array([x_t0,v_t0])
motion_t=np.zeros((len(motion_t0),len(time)))

## PID parameters (gain has to be larger since the step size is really small)
kp=2.5
ki=0.85
kd=1.5

integral=0   # initialize the integral
lasterr=0    # for the kd  



###########################################################

# System & environmental parameters

## Hose
H_l=8                  # length [m]
H_d=19.3e-3            # inner diameter[m]
L_1=8                  #  [m]
A_1=np.pi*(H_d/2)**2   # [m^2]
V_1=L_1*A_1                # [m^3] A_1*H_l  


## Fitting
F_l=49e-3                      # length [m]
F_d=11.9e-3                    # diameter [m]
A_2=np.pi*(F_d/2)**2           #[m^2]
V_2=A_2*F_l                    #[m^3] #A_2*F_l


## Main Valve
Kv=4.8 
Open_t=825e-3    # opening time [s]
Close_t=1700e-3  # closing time[s]
T_3=273          # [K]


## Nozzle
Th_d=9e-3                        # throat diameter [m]
Th_exit_d=11e-3                  # exit diameter [m]
A_throat=np.pi*(Th_d/2)**2      #[m^2]
A_e=np.pi*(Th_exit_d/2)**2      #[m^2]
epsilon=A_e/A_throat           #expansion ratio
T_4=273                        #[k]


###########################################################

## mass
m_hop=3.5    # mass of hopper [kg]
m_hose=1     # mass of hose   [kg]
prev_pose=x_t0  # for hose dynamic modeling
k_hose=6     # variable hopper mass [N/m]
F_RR=10      # rolling resistence [N]

## constant
R=8.314            # universal gas constant [J/kmol*K]
M=28.013           # nitrogen gas  [kg/kmol]
gamma=1.4          # ratio of specific heats (diatomic gas)
g=9.81             # [m/s^2]


f=0.0072                   #  f=0.316/Re^0.25  # darcy friction factor, assuming nitrogen gas flow rate of 1kg/s 
# Re=4*dotm/(\mu * pi* D(diameter))  dynamic viscosity is assume to be 1.76x10^-5 [Pa*s] 20 degree C and 1 atm 
lam_1=f*L_1/H_d             #lam_1=f*L_1/H_d
lam_2=lam_1
###########################################################
## to do list: correct the lam_1 lam_2 f values 

Me=1.84



In [ ]:

# MPC/Casadi test
x=ca.SX.sym('x')
v=ca.SX.sym('v')
states=ca.vertcat(x,v)

p3_u=ca.SX.sym('p3_u')
control=p3_u

# algebraic state
md4n=ca.SX.sym('md4n')
pe=ca.SX.sym('pe')
ve=ca.SX.sym('ve')
alg_states=ca.vertcat(md4n,pe,ve)

x_all=states
z_all=alg_states
p_all=control


md4n=A_throat*p3_u/(np.sqrt(T_4*R/(M*gamma)))*125/216  # m_choked
pe=p3_u/np.power(1+(0.2*Me**2), 3.5)                          #exit pressure
ve=np.sqrt((2*1.4*R*T_4)/0.4/M*(1-np.power(pe/p3_u,2/7)))    #exit velocity
alg=ca.vertcat(md4n,pe,ve)

# ode state
dotx=v
dotv=md4n*ve/3.5-9.81-10/3.5*np.sign(v) # need adding the effect of the rope later
ode=ca.vertcat(dotx,dotv)

dae={'x':x_all,'z':z_all,'p':p_all,'ode':ode,'alg':alg}
f=ca.Function('f',[x_all,z_all,p_all],[ode,alg],['x','z','p'],['ode','alg'])

# initial state
P_=[0.5]  # intial input
X_=[0.0,0.0] # initial state
Z_=[0.0,0.0,0.0] #initial algebraic state
XDOT_=[0.0,0.0]


res=f(p=P_,x=X_,z=Z_)

In [5]:
print(res['ode'])
print(res['alg'])

[0, -9.80999]
[2.41968e-06, 0.0818456, 15.1323]


In [10]:
J = f.factory('J', f.name_in(), ['jac:alg:z'])
res = J(p=P_, x=X_, z=Z_)
print(np.array(res["jac_alg_z"]))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [3]:
n_states=2
N=10
print(ca.DM.zeros((n_states * (N + 1))))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
N=5
n_control=2
state_0=np.array([0.1,0.2])

u0 = ca.DM.zeros((n_control, N))
X0 = ca.repmat(state_0, 1, N + 1)

print(u0)
print(X0)


[[0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0]]

[[0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
 [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]]
